In [1]:
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip3 install wbdata


In [3]:
import wbdata
import datetime

# Define the indicator for GDP growth (annual %)
indicators = {'NY.GDP.MKTP.KD.ZG': 'GDP growth (annual %)'}

# Set the date range
start_date = datetime.datetime(1987, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Retrieve GDP growth data for the world
gdp_growth_data = wbdata.get_dataframe(indicators, country='WLD', date=(start_date, end_date), freq='Y')

# Display the GDP growth data
print(gdp_growth_data)

# Save to CSV
gdp_growth_data.to_csv('/content/world_GDP_growth_data.csv')

      GDP growth (annual %)
date                       
2022               3.238021
2021               6.350090
2020              -2.878323
2019               2.675966
2018               3.282197
2017               3.451446
2016               2.813736
2015               3.128787
2014               3.131476
2013               2.872839
2012               2.691961
2011               3.336286
2010               4.517430
2009              -1.342370
2008               2.056153
2007               4.373197
2006               4.460593
2005               4.020487
2004               4.473970
2003               3.093669
2002               2.316450
2001               2.033625
2000               4.532918
1999               3.589073
1998               2.843122
1997               3.953367
1996               3.582286
1995               3.097579
1994               3.342330
1993               1.840448
1992               2.022340
1991               1.210701
1990               2.721270
1989               3

In [5]:
indicators = {'SL.UEM.TOTL.ZS': 'Unemployment Rate (%)'}

# Set the date range
start_date = datetime.datetime(1987, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Retrieve unemployment data for the world
unemployment_data = wbdata.get_dataframe(indicators, country='WLD', date=(start_date, end_date), freq='Y')

# Display the unemployment data
print(unemployment_data)

unemployment_data.to_csv('/content/world_unemployment_rates_data.csv')

      Unemployment Rate (%)
date                       
2022               5.260711
2021               6.064446
2020               6.608805
2019               5.598931
2018               5.768983
2017               5.907231
2016               5.978274
2015               6.006559
2014               5.965960
2013               6.107219
2012               6.159642
2011               6.156886
2010               6.286335
2009               6.419615
2008               5.832310
2007               5.777538
2006               5.976250
2005               6.227128
2004               6.337595
2003               6.495495
2002               6.428244
2001               6.202246
2000               6.123041
1999               6.288615
1998               6.170974
1997               6.022391
1996               5.992539
1995               5.930044
1994               5.800499
1993               5.559694
1992               5.290110
1991               5.133780
1990                    NaN
1989                

In [6]:
indicators={'FP.CPI.TOTL.ZG': 'Inflation'}

inflation_rates_data=wbdata.get_dataframe(indicators, country='WLD', date = ("1987-05-20", "2022-11-14"), freq='Y', source=None, parse_dates=False, keep_levels=False, skip_cache=False)
inflation_rates_data
inflation_rates_data.to_csv('/content/world_inflation_drates_ata.csv')

In [7]:
!pip3 install statsmodels

# Daily inflation from annual average

In [10]:
import numpy as np

def daily_inflation(start_date='1987-05-20', end_date='2022-11-14', annual_inflation_rates=None):
    # Generate daily date range
    dates = pd.date_range(start=start_date, end=end_date)
    daily_df = pd.DataFrame({'Date': dates})

    # Ensure the 'Year' column is an integer
    annual_inflation_rates['Year'] = annual_inflation_rates['date'].astype(int)
    annual_inflation_rates = annual_inflation_rates.rename(columns={'Inflation': 'Annual_Inflation_Rate'})

    # Calculate daily inflation rate
    inflation_data = []
    for _, row in annual_inflation_rates.iterrows():
        daily_rate = (1 + row['Annual_Inflation_Rate']) ** (1 / 365) - 1
        inflation_data.append({'Year': row['Year'], 'Daily_Inflation_Rate': daily_rate})

    inflation_df = pd.DataFrame(inflation_data)

    # Map each date to its corresponding annual daily inflation rate
    daily_df['Year'] = daily_df['Date'].dt.year
    daily_df = daily_df.merge(inflation_df, on='Year', how='left')

    # Calculate the cumulative inflation factor
    daily_df['Cumulative_Inflation_Factor'] = (1 + daily_df['Daily_Inflation_Rate']).cumprod()

    # Select relevant columns
    daily_df = daily_df[['Date', 'Cumulative_Inflation_Factor','Daily_Inflation_Rate']]
    return daily_df

# Load annual inflation rates data from CSV
annual_inflation_rates = pd.read_csv("/content/world_inflation_drates_ata.csv")

# Generate daily inflation factors
daily_inflation_df = daily_inflation(annual_inflation_rates=annual_inflation_rates)
daily_inflation_df.to_csv("/content/world_inflation_drates_ata.csv")
print(daily_inflation_df)

            Date  Cumulative_Inflation_Factor  Daily_Inflation_Rate
0     1987-05-20                 1.005229e+00              0.005229
1     1987-05-21                 1.010485e+00              0.005229
2     1987-05-22                 1.015769e+00              0.005229
3     1987-05-23                 1.021081e+00              0.005229
4     1987-05-24                 1.026420e+00              0.005229
...          ...                          ...                   ...
12958 2022-11-10                 5.001909e+25              0.006017
12959 2022-11-11                 5.032004e+25              0.006017
12960 2022-11-12                 5.062280e+25              0.006017
12961 2022-11-13                 5.092738e+25              0.006017
12962 2022-11-14                 5.123380e+25              0.006017

[12963 rows x 3 columns]


# Daily GDP from annual

In [11]:
def daily_inflation(start_date='1987-05-20', end_date='2022-11-14', annual_gdp_growth_rate=None):
    # Generate daily date range
    dates = pd.date_range(start=start_date, end=end_date)
    daily_df = pd.DataFrame({'Date': dates})

    # Ensure the 'Year' column is an integer
    annual_gdp_growth_rate['Year'] = annual_gdp_growth_rate['date'].astype(int)
    annual_gdp_growth_rate = annual_gdp_growth_rate.rename(columns={'GDP growth (annual %)': 'Annual_GDP_Growth_Rate'})

    # Calculate daily inflation rate
    gdp_growth_data = []
    for _, row in annual_gdp_growth_rate.iterrows():
        daily_rate = (1 + row['Annual_GDP_Growth_Rate']) ** (1 / 365) - 1
        gdp_growth_data.append({'Year': row['Year'], 'Daily_GDP_Growth_Rate': daily_rate})

    inflation_df = pd.DataFrame(gdp_growth_data)

    # Map each date to its corresponding annual daily inflation rate
    daily_df['Year'] = daily_df['Date'].dt.year
    daily_df = daily_df.merge(inflation_df, on='Year', how='left')

    # Calculate the cumulative inflation factor
    daily_df['Cumulative_GDP_Growth_Factor'] = (1 + daily_df['Daily_GDP_Growth_Rate']).cumprod()

    # Select relevant columns
    daily_df = daily_df[['Date', 'Cumulative_GDP_Growth_Factor','Daily_GDP_Growth_Rate']]
    return daily_df

# Load annual inflation rates data from CSV
annual_gdp_growth_rate = pd.read_csv("/content/world_GDP_growth_data.csv")

# Generate daily inflation factors
daily_gdp_df = daily_inflation(annual_gdp_growth_rate=annual_gdp_growth_rate)
daily_gdp_df.to_csv("/content/world_daily_GDP_growth_data.csv")
print(daily_inflation_df)

            Date  Cumulative_Inflation_Factor  Daily_Inflation_Rate
0     1987-05-20                 1.005229e+00              0.005229
1     1987-05-21                 1.010485e+00              0.005229
2     1987-05-22                 1.015769e+00              0.005229
3     1987-05-23                 1.021081e+00              0.005229
4     1987-05-24                 1.026420e+00              0.005229
...          ...                          ...                   ...
12958 2022-11-10                 5.001909e+25              0.006017
12959 2022-11-11                 5.032004e+25              0.006017
12960 2022-11-12                 5.062280e+25              0.006017
12961 2022-11-13                 5.092738e+25              0.006017
12962 2022-11-14                 5.123380e+25              0.006017

[12963 rows x 3 columns]


# Daily unemployment rate from annual

In [12]:
def daily_unemployment(start_date='1987-05-20', end_date='2022-11-14', annual_unemployment_rate=None, lag_days=30, rolling_window=90):

    # Generate a daily date range
    dates = pd.date_range(start=start_date, end=end_date)
    daily_df = pd.DataFrame({'Date': dates})

    # Ensure the 'Year' column is an integer and rename columns
    annual_unemployment_rate = annual_unemployment_rate.rename(columns={'date': 'Year', 'Unemployment Rate (%)': 'Annual_Unemployment_Rate'})
    annual_unemployment_rate['Year'] = pd.to_datetime(annual_unemployment_rate['Year'].astype(str) + '-01-01')

    # Expand to a daily frequency and use interpolation
    unemployment_series = annual_unemployment_rate.set_index('Year').resample('D').interpolate(method='linear')
    unemployment_series = unemployment_series.rename(columns={'Annual_Unemployment_Rate': 'Daily_Unemployment_Rate'})

    # Merge with the daily date range to match the specified date range
    daily_df = daily_df.merge(unemployment_series, left_on='Date', right_index=True, how='left')

    # Calculate lagged and rolling unemployment features
    daily_df['Lagged_Unemployment_Rate'] = daily_df['Daily_Unemployment_Rate'].shift(lag_days)
    daily_df['Rolling_Unemployment_Rate'] = daily_df['Daily_Unemployment_Rate'].rolling(window=rolling_window, min_periods=1).mean()

    # Select relevant columns
    daily_df = daily_df[['Date', 'Daily_Unemployment_Rate', 'Lagged_Unemployment_Rate', 'Rolling_Unemployment_Rate']]
    return daily_df

# Load annual unemployment data from CSV
annual_unemployment_rate = pd.read_csv("/content/world_unemployment_rates_data.csv")


# Generate daily unemployment rates with lag and rolling features
daily_unemployment_df = daily_unemployment(annual_unemployment_rate=annual_unemployment_rate)
daily_unemployment_df.to_csv("/content/world_daily_unemployment_rate_data.csv", index=False)
print(daily_unemployment_df)

            Date  Daily_Unemployment_Rate  Lagged_Unemployment_Rate  \
0     1987-05-20                      NaN                       NaN   
1     1987-05-21                      NaN                       NaN   
2     1987-05-22                      NaN                       NaN   
3     1987-05-23                      NaN                       NaN   
4     1987-05-24                      NaN                       NaN   
...          ...                      ...                       ...   
12958 2022-11-10                      NaN                       NaN   
12959 2022-11-11                      NaN                       NaN   
12960 2022-11-12                      NaN                       NaN   
12961 2022-11-13                      NaN                       NaN   
12962 2022-11-14                      NaN                       NaN   

       Rolling_Unemployment_Rate  
0                            NaN  
1                            NaN  
2                            NaN  
3      